In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_excel("Фасады.xlsx", sheet_name="Фасады_Секции по файлу")
data

,Сделано в tangl,tangl.cloud,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Проект:,"ЖК ""Пушкино"" (Омск)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Модель:,OMSK-PKN_GP1.1_Фасады,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Версия и дата:,V11.2 - 30.07.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Утвержденная версия:,Нет утвержденной версии,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Результаты анализа - cводное дерево: Фасады_Се...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36944,1.9.1.1.4.5,2. Работы строительно-монтажные NEW,004.014.033.004.005,8233427,8233427 - PS_Наружная_Отделка_Штукатурка_Marsh...,NaN,м2,2.3055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36945,1.9.1.1.4.5,2. Работы строительно-монтажные NEW,004.014.033.004.005,8250727,8250727 - PS_Наружная_Отделка_Штукатурка_Marsh...,NaN,м2,2.081511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36946,1.9.1.1.4.5,2. Работы строительно-монтажные NEW,004.014.033.004.005,8262856,8262856 - PS_Наружная_Отделка_Штукатурка_Marsh...,NaN,м2,0.342159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36947,1.9.1.1.4.5,2. Работы строительно-монтажные NEW,004.014.033.004.005,8263228,8263228 - PS_Наружная_Отделка_Штукатурка_Marsh...,NaN,м2,0.342,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. Clear data from the tangle. 

In [2]:
#=== Clean data
df = data[['Unnamed: 2', 'Unnamed: 3']]
df = df.dropna()

#=== Filter  
df = df[df['Unnamed: 2'].str .count(r"\.") > 1]

#=== Rename column 
df.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
df = df.reset_index(drop=True)
df['ElementId'] = df['ElementId'].astype('int64')
tangle_data = df
tangle_data

,Position,ElementId
0,004.014.033.001.002,588239
1,004.014.033.001.002,588240
2,004.014.033.001.002,588241
3,004.014.033.001.002,588242
4,004.014.033.001.002,588243
...,...,...
24369,004.014.033.004.005,8233323
24370,004.014.033.004.005,8233427
24371,004.014.033.004.005,8250727
24372,004.014.033.004.005,8262856


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
df_groups = df.groupby('ElementId')['Position'].apply(list).reset_index()

mlb = MultiLabelBinarizer()
y1 = mlb.fit_transform(df_groups['Position'])
y_df11 = pd.DataFrame(y1, columns=mlb.classes_,index=df_groups['ElementId'])
y_df11 = y_df11.add_prefix('Target_')
df_groups

,ElementId,Position
0,532164,"[004.014.033.003.002, 004.014.033.003.002]"
1,532168,"[004.014.033.003.002, 004.014.033.003.002]"
2,532170,"[004.014.033.003.002, 004.014.033.003.002]"
3,532172,"[004.014.033.003.002, 004.014.033.003.002]"
4,532286,"[004.014.033.003.002, 004.014.033.003.002]"
...,...,...
4085,8599899,"[004.014.033.001.005, 004.014.033.001.008, 004..."
4086,8600494,"[004.014.033.001.005, 004.014.033.001.008, 004..."
4087,8600572,"[004.014.033.001.005, 004.014.033.001.008, 004..."
4088,8601330,"[004.014.033.001.005, 004.014.033.001.008, 004..."


## 2. Clear data from Revit - Вернусь потом, думаю что сперва стоит отобрать элементы по id каторые мне нужны иначе я заструлюсь все чистить

In [ ]:
revit_df = pd.read_csv("revit_params_export.csv", sep=',')
revit_df

C:\Users\homer\AppData\Local\Temp\ipykernel_18976\3656291552.py:1: DtypeWarning: Columns (142,143,156,164,167,170,175,177,184,185,187,188,189,190,192,198,200,202,208,216,220,244,294,341,417,421,465,607,704,705,706,707,708,709,710,711,712,713,714,715,735,744,745,746,747,748,752,753,754,755,756,757,758,759,793,794,796,822,823,881,892,893,894,895,939,1054,1063,1077,1084,1085,1086,1087,1088,1089,1090,1091,1112,1119,1221,1316,1640,1643,1647,1650,1673,1712,1715,1720,1834,1837,1903,1950) have mixed types. Specify dtype option on import or set low_memory=False.
  revit_df = pd.read_csv("revit_params_export.csv", sep=',')


,#05#Размер.X,#05_ширины_свободной_зоны,#_Вниз слева,#_Вниз справа,#_Высота двери,#_Высота окна,#_Высота окна в свету,#_Высота отделки,#_Высота откоса двери,#_Высота откоса низ,...,▂ ▃ ▄ ▅ ▆ ▇ Материал 14 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 15 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 2 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 3 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 4 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 5 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 6 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 7 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 8 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 9 ▇ ▆ ▅ ▄ ▃ ▂
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
test_revit_df = revit_df[revit_df['Category'] == 'Стены']
test_revit_df

,#05#Размер.X,#05_ширины_свободной_зоны,#_Вниз слева,#_Вниз справа,#_Высота двери,#_Высота окна,#_Высота окна в свету,#_Высота отделки,#_Высота откоса двери,#_Высота откоса низ,...,▂ ▃ ▄ ▅ ▆ ▇ Материал 14 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 15 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 2 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 3 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 4 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 5 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 6 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 7 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 8 ▇ ▆ ▅ ▄ ▃ ▂,▂ ▃ ▄ ▅ ▆ ▇ Материал 9 ▇ ▆ ▅ ▄ ▃ ▂
11535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Check a mixed type columns
column_names = revit_df.columns.tolist()
mixed_cols_indices = [
    142,143,156,164,167,170,175,177,184,185,187,188,189,190,192,198,200,202,208,216,
    220,244,294,341,417,421,465,607,704,705,706,707,708,709,710,711,712,713,714,715,
    735,744,745,746,747,748,752,753,754,755,756,757,758,759,793,794,796,822,823,892,
    893,894,895,939,1054,1063,1077,1084,1085,1086,1087,1088,1089,1090,1091,1112,1119,
    1221,1316,1640,1643,1647,1650,1673,1712,1715,1720,1834,1837,1903,1950
]
problematic_columns = [column_names[i] for i in mixed_cols_indices if i < len(column_names)]
problematic_columns

['ADSK_URL документации изделия',
 'ADSK_URL страницы изделия',
 'ADSK_Группирование',
 'ADSK_Единица измерения',
 'ADSK_Зона',
 'ADSK_Код изделия',
 'ADSK_Марка',
 'ADSK_Марка конструкции',
 'ADSK_Материал наименование',
 'ADSK_Материал обозначение',
 'ADSK_Метка основы',
 'ADSK_Наименование',
 'ADSK_Наименование и номер цвета',
 'ADSK_Наименование краткое',
 'ADSK_Наименование элемента фасада',
 'ADSK_Номер секции',
 'ADSK_Обозначение',
 'ADSK_Основной материал элемента',
 'ADSK_Отверстие_Функция',
 'ADSK_Позиция',
 'ADSK_Примечание',
 'ADSK_Этаж',
 'PS_Отметка перемычки',
 'URL',
 'Аналитическая конструкция',
 'Арматурный стержень',
 'Внизу настройки для изогнутой арматуры',
 'Группа модели',
 'Заполнение 1.1',
 'Заполнение 1.2',
 'Заполнение 1.3',
 'Заполнение 1.4',
 'Заполнение 2.1',
 'Заполнение 2.2',
 'Заполнение 2.3',
 'Заполнение 2.4',
 'Заполнение 3.1',
 'Заполнение 3.2',
 'Заполнение 3.3',
 'Заполнение 3.4',
 'Изготовитель',
 'Имя вида',
 'Имя материала 1',
 'Имя материала 1

Filter revit data 

In [ ]:
import re
mask = revit_df.columns.str.contains(r"[^a-zA-Zа-яА-Я0-9 _'()/.№]", na=False)
# f_filter_revit_df = revit_df.loc[:,~mask]
f_filter_revit_df = revit_df.loc[:,~mask]


Список категорий которые точно не нужны:<br>
'Фасады', 'Варианты нагружений на конструкцию', 'Площадки','Уклоны в точках','Фрагмент плана','Элементы узлов','Марки помещений','Автонанесение размеров в эскизе'

In [68]:
cat_to_drob = ['Фасады', 'Варианты нагружений на конструкцию', 'Площадки','Уклоны в точках','Фрагмент плана','Элементы узлов','Марки помещений','Автонанесение размеров в эскизе']
drob_mask = ~f_filter_revit_df['Category'].isin(cat_to_drob)
f_filter_revit_df = f_filter_revit_df[drob_mask]
f_filter_revit_df

,01 этаж,02 этаж,03 этаж,1,1 перемычка,1 сегмент,1/4 по глубине,1/4 по длине,10,11,...,яОткосНижний,яОткосПравый,яТолщина откоса расч,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),яШаг лапок L1,яШаг лапок L2,яШирина 1 ряда,яШирина 2 ряда,яШирина 3 ряда
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
f_filter_revit_df['ADSK_Группирование']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
13681    NaN
13682    NaN
13683    NaN
13684    NaN
13685    NaN
Name: ADSK_Группирование, Length: 9180, dtype: object

Теперь разобью по группам(по катагории) и отсею лишние параметры, потом соединю 

Список параметров который надо удалить:
- PS_Секция: Не даст велью
- стадия
- Рабочий набор 
- ADSK_Номер секции
- стоимость 
- Изображение типоразмера
- PS_Вложение (вкл/выкл)
- PS_Тип вложения
- Спецификация/Тип
- Шероховатость
- Нейтральная шина
- Номинал нейтральной
- Шина заземления

> Думаю стоит взять все категории, удалить пустные колонки, а затем сделать анализ стат значимотси для каждого таргета

In [85]:
f_filter_revit_df.groupby('Category').size().sort_values(ascending=False).keys()

Index(['Стены', 'Обобщенные модели', 'Каркас несущий', 'Окна',
       'Импосты витража', 'Панели витража', 'Материалы', 'Двери',
       'Схемы разрезки витражей', 'Перекрытия', 'Сантехнические приборы',
       'Ограждение', 'Балясины', 'Соединения несущих конструкций',
       'Электрические приборы', 'Электрооборудование', 'Оборудование', 'Марши',
       'Лестницы', 'Сегменты труб', 'Несущая арматура',
       'Специальное оборудование', 'Ребра плит', 'Проемы для шахты', 'Части'],
      dtype='object', name='Category')

In [97]:
test_data = f_filter_revit_df[f_filter_revit_df['Category'] == 'Стены']
test_data

,01 этаж,02 этаж,03 этаж,1,1 перемычка,1 сегмент,1/4 по глубине,1/4 по длине,10,11,...,яОткосНижний,яОткосПравый,яТолщина откоса расч,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),яШаг лапок L1,яШаг лапок L2,яШирина 1 ряда,яШирина 2 ряда,яШирина 3 ряда
11535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.1 Сопоставление датафреймов

In [5]:
revit_df = pd.read_csv("revit_params_OMSK-PKN_GP1.1_01_AR_отсоединено.csv", sep=',')
revit_df['ElementId'] = revit_df['ElementId'].astype('int64')
import re
mask = revit_df.columns.str.contains(r"[^a-zA-Zа-яА-Я0-9 _'()/.№]", na=False)
f_filter_revit_df1 = revit_df.loc[:,~mask]
# f_filter_revit_df = f_filter_revit_df.set_index('ElementId')
f_filter_revit_df1


C:\Users\homer\AppData\Local\Temp\ipykernel_1200\3235677437.py:1: DtypeWarning: Columns (142,143,156,164,167,170,175,177,184,185,187,188,189,190,192,198,200,202,208,216,220,244,294,341,417,421,465,607,704,705,706,707,708,709,710,711,712,713,714,715,735,744,745,746,747,748,752,753,754,755,756,757,758,759,793,794,796,822,823,881,892,893,894,895,939,1054,1063,1077,1084,1085,1086,1087,1088,1089,1090,1091,1112,1119,1221,1316,1640,1643,1647,1650,1673,1712,1715,1720,1834,1837,1903,1950) have mixed types. Specify dtype option on import or set low_memory=False.
  revit_df = pd.read_csv("revit_params_OMSK-PKN_GP1.1_01_AR_отсоединено.csv", sep=',')


,01 этаж,02 этаж,03 этаж,1,1 перемычка,1 сегмент,1/4 по глубине,1/4 по длине,10,11,...,яОткосНижний,яОткосПравый,яТолщина откоса расч,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),яШаг лапок L1,яШаг лапок L2,яШирина 1 ряда,яШирина 2 ряда,яШирина 3 ряда
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
revit_df = pd.read_csv("revit_params_OMSK-PKN_GP1.1_02,08_AR_отсоединено.csv", sep=',')
revit_df['ElementId'] = revit_df['ElementId'].astype('int64')
import re
mask = revit_df.columns.str.contains(r"[^a-zA-Zа-яА-Я0-9 _'()/.№]", na=False)
f_filter_revit_df2 = revit_df.loc[:,~mask]
# f_filter_revit_df = f_filter_revit_df.set_index('ElementId')
f_filter_revit_df2

C:\Users\homer\AppData\Local\Temp\ipykernel_1200\1882363526.py:1: DtypeWarning: Columns (149,150,163,171,174,177,182,184,191,192,194,195,196,197,199,205,207,209,215,223,227,251,296,331,381,547,643,652,653,654,655,656,660,661,662,663,664,665,666,667,695,696,698,722,723,783,784,785,924,932,945,952,953,954,955,956,957,958,959,976,981,1159,1431,1434,1438,1441,1459,1496,1497,1710) have mixed types. Specify dtype option on import or set low_memory=False.
  revit_df = pd.read_csv("revit_params_OMSK-PKN_GP1.1_02,08_AR_отсоединено.csv", sep=',')


,01 этаж,02 этаж,03 этаж,1,10,11,12,13,14,15,...,яНахлестОтделкиЛево,яНахлестОтделкиНиз,яНахлестОтделкиПраво,яНачало вырезов,яОткосВерхний,яОткосЛевый,яОткосНижний,яОткосПравый,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df1 = pd.DataFrame({'A': ['5', '87', '42', '5']})

df2 = pd.DataFrame({'A': ['5', '3', '87', '33'],
                     'B1': ['B0', 'B1', 'B5', 'B3'],
                     'C1': ['C4', 'C5', 'C6', 'C7'],
                     'D1': ['D4', 'D5', 'D6', 'D7']})

df1.merge(df2, left_on='A', right_on='A')

,A,B1,C1,D1
0,5,B0,C4,D4
1,87,B5,C6,D6
2,5,B0,C4,D4


In [7]:
pd.concat([f_filter_revit_df1,f_filter_revit_df2],sort=False, axis=0)

,01 этаж,02 этаж,03 этаж,1,1 перемычка,1 сегмент,1/4 по глубине,1/4 по длине,10,11,...,Четверть Левая_видимость,Четверть Правая,Четверть Правая_видимость,Четверть_Верхняя_итог,Четверть_Левая_итог,Четверть_Правая_итог,Ширина компенсатора,Ширина проема внизу,яВылет компенсатора 1,яВылет компенсатора 2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dict_obj = {}
for key, value in f_filter_revit_df.dtypes.items():
    if value is not np.dtype('float64'):
        dict_obj[key] = value

len(dict_obj)
# dict_obj

NameError: name 'f_filter_revit_df' is not defined

Функция объединения всех датафреймов

In [11]:
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

revit_data = multi_concat()
# revit_data.set_index('ElementId')

C:\Users\homer\AppData\Local\Temp\ipykernel_1200\3857291529.py:8: DtypeWarning: Columns (142,143,156,164,167,170,175,177,184,185,187,188,189,190,192,198,200,202,208,216,220,244,294,341,417,421,465,607,704,705,706,707,708,709,710,711,712,713,714,715,735,744,745,746,747,748,752,753,754,755,756,757,758,759,793,794,796,822,823,881,892,893,894,895,939,1054,1063,1077,1084,1085,1086,1087,1088,1089,1090,1091,1112,1119,1221,1316,1640,1643,1647,1650,1673,1712,1715,1720,1834,1837,1903,1950) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\homer\AppData\Local\Temp\ipykernel_1200\3857291529.py:8: DtypeWarning: Columns (149,150,163,171,174,177,182,184,191,192,194,195,196,197,199,205,207,209,215,223,227,251,296,331,381,547,643,652,653,654,655,656,660,661,662,663,664,665,666,667,695,696,698,722,723,783,784,785,924,932,945,952,953,954,955,956,957,958,959,976,981,1159,1431,1434,1438,1441,1459,1496,1497,1710) have mixed types. Specify dtype optio

In [10]:
del revit_data['ElementId']

In [72]:
revit_data['ElementId'].values

array([7579436, 7579438, 7579465, ..., 8746237, 8746320, 8746412],
      dtype=int64)

In [62]:
revit_data['ADSK_Группирование'].value_counts()

ADSK_Группирование
Жилье                        3317
Фасад                        2108
Вентшахты                     986
МОП                           449
Ниже отм. 0,000               409
Парапет                       322
Балкон                        239
Балконы                       112
Техническое                    84
Тамбур                         77
Ритейл                         54
Деф.шов                        48
Кровля                         48
Вентшахта                      44
терраса                        39
Откосы                         29
Выход на кровлю                25
кирпичная стена на фасаде      25
отмостка                       21
Фасад_Лоджия                   20
Фасад_Кирпич                   13
Люк                            12
Люк Вентшахт                   10
Элементы подвала                5
Вентшахты_Люк                   4
Крышка                          4
Вентшахты_люк                   4
ОШИБКА                          4
Спец                         

# Шаг 3. Обор признаков
Я пытался как-то статистически или вручную отобрать признаки, но пропуски это норма для такой задачи, у части эелментов модели просто не существует части признаков.<br>
Остановился на решении, что буду отбирать признаки через деревянные модели. l1 испольщовать смысла нет, так как много пропусков, нет линейной зависимости между переменными.

Обьединение датафреймов по id - По сути не надо. Удалил))))
Бля, вернул. Как ка кнужно лоя сокращения данных

In [12]:
# marged_df = y_df.join(revit_data, how='left')
marged_df = df_groups.merge(revit_data, left_on='ElementId', right_on='ElementId')
marged_df

,ElementId,Position,#05#Размер.X,#05_ширины_свободной_зоны,#_Вниз слева,#_Вниз справа,#_Высота двери,#_Высота окна,#_Высота окна в свету,#_Высота отделки,...,Минимум,Основная часть,Отступ полки,Проступь + 20,Сливная часть,УголПоворотаВерха,УголПоворотаТрубы,Число проступей,Ширина B,Ширина проступи
0,532372,[004.014.033.003.002],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,559789,"[004.014.033.001.002, 031.003.002.002, 004.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,560150,"[004.014.033.001.002, 031.003.002.002, 004.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,560198,"[004.014.033.001.002, 031.003.002.002, 004.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,560275,"[004.014.033.001.002, 031.003.002.002, 004.014...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4742,8599899,"[004.014.033.001.005, 004.014.033.001.008, 004...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4743,8600494,"[004.014.033.001.005, 004.014.033.001.008, 004...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4744,8600572,"[004.014.033.001.005, 004.014.033.001.008, 004...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4745,8601330,"[004.014.033.001.005, 004.014.033.001.008, 004...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
marged_df['ADSK_Группирование'].value_counts()

ADSK_Группирование
Фасад              2057
Вентшахты           529
Откосы               29
Выход на кровлю      25
Фасад_Лоджия         20
Фасад_Кирпич          7
терраса               5
Парапет               2
Ритейл                1
Name: count, dtype: int64

Идея в том, что я ДЛЯ КАЖДОГО таргета найду самые важные для него признаки и обьединю потом из все, для мультилейблинга

In [85]:
marged_df.select_dtypes('object').columns

Index(['Position', 'ADSK_URL документации изделия',
       'ADSK_URL страницы изделия', 'ADSK_Группирование',
       'ADSK_Единица измерения', 'ADSK_Зона', 'ADSK_Код изделия', 'ADSK_Марка',
       'ADSK_Марка конструкции', 'ADSK_Материал наименование',
       ...
       'Уголок неравнополочный', 'Уголок равнополочный', 'Швеллер паралл',
       'Шрифт текста', 'Имя оси 1', 'Имя оси 2', 'Имя оси 3',
       'Текст в выноске', 'Разработчик', 'Таблица'],
      dtype='object', length=101)

Разделение дарафрейма

In [13]:
target_cols = ['ElementId', 'Position']
X_df = marged_df.drop(columns=target_cols, axis=1)
y_df = marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y_df['Position'])
y_df = pd.DataFrame(y, columns=mlb.classes_,index=y_df['ElementId'])
y_df = y_df.add_prefix('Target_')
y_df

,Target_004.014.033.001.002,Target_004.014.033.001.005,Target_004.014.033.001.008,Target_004.014.033.001.012,Target_004.014.033.001.013,Target_004.014.033.002.001,Target_004.014.033.002.002,Target_004.014.033.002.003,Target_004.014.033.002.004,Target_004.014.033.003.002,...,Target_021.005.003.005.051,Target_021.005.003.005.052,Target_021.005.003.005.053,Target_021.005.003.005.054,Target_021.005.003.005.059,Target_021.005.003.005.063,Target_031.003.002.001,Target_031.003.002.002,Target_031.003.002.003,Target_031.006.003
ElementId,,,,,,,,,,,,,,,,,,,,,
532372,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
559789,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
560150,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
560198,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
560275,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8599899,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8600494,0,1,1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8600572,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# import lightgbm as lgb
# # подготовка
# target_cols = [col for col in marged_df.columns if col.startswith('Target_')]
# X_df = marged_df.drop(columns=target_cols, axis=1)
# y_df = marged_df[target_cols]

constant_cols = X_df.columns[X_df.nunique() <= 1]
X_df = X_df.drop(columns=constant_cols)
print("Константные колонки:", len(list(constant_cols)))
X_df


Константные колонки: 2262


,ADSK_Группа конструкций,ADSK_Группирование,ADSK_Зона,ADSK_Материал,ADSK_Наименование,ADSK_Номер секции,ADSK_Основной материал элемента,ADSK_Примечание,ADSK_Размер_Высота,ADSK_Размер_Толщина,...,Тип,Толщина утеплителя углубления,Углубление сверху,Углубление слева,Углубление снизу,Углубление справа,Уровень,Функция,Цвет штриховки при низкой детализации,Штриховка при низкой детализации
0,NaN,NaN,NaN,NaN,"Стена из силикатного блока, t=250","ГП1.1-С2, ГП1.1-С8 (типовая)",NaN,NaN,NaN,0.000000,...,521665.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10526880.0,19.0
1,NaN,Фасад,NaN,NaN,NaN,ГП1.1-С3,NaN,NaN,NaN,0.492126,...,559382.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12177407.0,19.0
2,NaN,Фасад,NaN,NaN,NaN,"ГП1.1-С3, ГП1.1-С5, ГП1.1-С7 (типовая)",NaN,NaN,NaN,0.492126,...,559382.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12177407.0,19.0
3,NaN,Фасад,NaN,NaN,NaN,"ГП1.1-С3, ГП1.1-С5, ГП1.1-С7 (типовая)",NaN,NaN,NaN,0.492126,...,559382.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12177407.0,19.0
4,NaN,Фасад,NaN,NaN,NaN,"ГП1.1-С3, ГП1.1-С5, ГП1.1-С7 (типовая)",NaN,NaN,NaN,0.492126,...,559382.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12177407.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,8637743.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12632256.0,19.0
4743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,8730062.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12632256.0,19.0
4744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,8637743.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12632256.0,19.0
4745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,8730062.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12632256.0,19.0


исправление - LightGBMError: Feature (Вставка_Четверть_Глубина) appears more than one time

In [15]:
cat_cols = X_df.select_dtypes('category').columns
for col in cat_cols:
    print(f"Колонка '{col}': {X_df[col].nunique()} уникальных категорий")

In [20]:
import lightgbm as lgb
# замена object на str
cat_features = []
for col in X_df.select_dtypes('object').columns:
    X_df[col] = X_df[col].astype('category')
    cat_features.append(col)
# проверка
selected_feature = []
for target in y_df:
    model = lgb.LGBMClassifier(
        max_depth=5,
        objective='binary'
    )

    model.fit(X_df, y_df[target])
    selected_feature.append(model.feature_importances_)

selected_feature

[array([  0,   0,   0,   0,   0,   0,   0,   0,   0, 352,   0,   0,   0,
         89,   9,  41,   2,   4,  76,   0,   9,   6,  34,   1,   0,   0,
          0,  29,  75,   0,   0,   0, 122,   0,   0,   0,   0,   0,   5,
         17,   3,   0,   7,   0,   0,   0, 219, 130,   0,   0, 178,  38,
          6,   3,   0,   0,   0,   0,   0,   0,   0,   0,  44,   0]),
 array([  0,  33,   0,   0,  25,   0,  19,   0,   1, 190,   0,   0,   0,
         25, 120,  19,   0,  17,   7,   1,   6,  88,   7,  11,  10,   0,
          7,  19,  34,  13,  18,   0, 152,   0,   0,   0,   0,   0,   0,
         73,   0,   7,   9,  76,   0,   0, 240, 106,   0,  31, 121, 100,
          5,  10,   0,   0,   0,   0,   0,   0,  45,   0,  58,   0]),
 array([  0,  33,   0,   0,  25,   0,  19,   0,   1, 190,   0,   0,   0,
         25, 120,  19,   0,  17,   7,   1,   6,  88,   7,  11,  10,   0,
          7,  19,  34,  13,  18,   0, 152,   0,   0,   0,   0,   0,   0,
         73,   0,   7,   9,  76,   0,   0, 240, 106,   0,

# Pipline

In [279]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

data = pd.read_excel("Фасады.xlsx", sheet_name="Фасады_Секции по файлу")
data

#=== Clean data
df = data[['Unnamed: 2', 'Unnamed: 3']]
df = df.dropna()

df = df[df['Unnamed: 2'].str.count(r"\.") > 1]

df.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
df = df.reset_index(drop=True)
df['ElementId'] = df['ElementId'].astype('int64')
tangle_data = df

#==== Concat all csv
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_OM" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

revit_data = multi_concat()

#=== Marge
df_groups = df.groupby('ElementId')['Position'].apply(list).reset_index()
marged_df = df_groups.merge(revit_data, left_on='ElementId', right_on='ElementId')


C:\Users\homer\AppData\Local\Temp\ipykernel_21520\937192208.py:29: DtypeWarning: Columns (142,143,156,164,167,170,175,177,184,185,187,188,189,190,192,198,200,202,208,216,220,244,294,341,417,421,465,607,704,705,706,707,708,709,710,711,712,713,714,715,735,744,745,746,747,748,752,753,754,755,756,757,758,759,793,794,796,822,823,881,892,893,894,895,939,1054,1063,1077,1084,1085,1086,1087,1088,1089,1090,1091,1112,1119,1221,1316,1640,1643,1647,1650,1673,1712,1715,1720,1834,1837,1903,1950) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\homer\AppData\Local\Temp\ipykernel_21520\937192208.py:29: DtypeWarning: Columns (149,150,163,171,174,177,182,184,191,192,194,195,196,197,199,205,207,209,215,223,227,251,296,331,381,547,643,652,653,654,655,656,660,661,662,663,664,665,666,667,695,696,698,722,723,783,784,785,924,932,945,952,953,954,955,956,957,958,959,976,981,1159,1431,1434,1438,1441,1459,1496,1497,1710) have mixed types. Specify dtype opt

In [283]:
#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
X_df = marged_df.drop(columns=target_cols, axis=1)
y_df = marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
fit_transform = mlb.fit_transform(y_df['Position'])
y_df = pd.DataFrame(fit_transform, columns=mlb.classes_,index=y_df['ElementId'])
y_df = y_df.add_prefix('Target_')

constant_cols = X_df.columns[X_df.nunique() <= 1]
X_df = X_df.drop(columns=constant_cols)
print("Константные колонки:", len(list(constant_cols)))

X_df.fillna("NAN", inplace=True)

Константные колонки: 2262


C:\Users\homer\AppData\Local\Temp\ipykernel_21520\513930619.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NAN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X_df.fillna("NAN", inplace=True)


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

lgbm = LGBMClassifier(
    random_state=42
)

catb = CatBoostClassifier(
    random_state=42,
    nan_mode="Min"
)

multi_lgbm = MultiOutputClassifier(lgbm)

cat_features = []
for col in X_df.select_dtypes('object').columns:
    X_df[col] = X_df[col].astype(str)
    cat_features.append(col)
    X_df[col] = X_df[col].astype('category')

multi_catb = MultiOutputClassifier(catb)


#=== обучение
print("Обучение LightGBM")
multi_lgbm.fit(X_df, y_df)

print("Обучение CatBoost")
multi_catb.fit(X_df, y_df, cat_features=cat_features)

In [308]:
X_df['ADSK_Материал']

0       NAN
1       NAN
2       NAN
3       NAN
4       NAN
       ... 
4742    NAN
4743    NAN
4744    NAN
4745    NAN
4746    NAN
Name: ADSK_Материал, Length: 4747, dtype: category
Categories (3, object): ['4876628.0', '6603050.0', 'NAN']

In [293]:
import joblib
import json

# Сохранение моделей, cat_features и фичей
# joblib.dump(multi_lgbm, 'lgbm.pkl')
# joblib.dump(multi_catb, 'catb.pkl')

with open('features.json', 'w') as f:
    json.dump(X_df.columns.to_list(), f)

print("Модели и метаданные сохранены.")

Модели и метаданные сохранены.


In [ ]:
with open('cat_features.json', 'w') as f:
    json.dump(cat_features, f)

In [13]:
with open('features.json', 'r') as f:
    features = json.load(f)


In [ ]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

#=== Предсказание вероятностей
y_pred_proba_lgbm = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_catb = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_catb.estimators_]).T

y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

#=== отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_lgbm > 0.5).astype(int)

In [16]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score

#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(y_df, y_pred_ensemble, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(y_df, y_pred_ensemble)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(y_df, y_pred_ensemble, average='macro')
f1_micro = f1_score(y_df, y_pred_ensemble, average='micro')

precision_macro = precision_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
precision_micro = precision_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

recall_macro = recall_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
recall_micro = recall_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

#=== Метрики
print("Результаты ансамбля LGBM + CatBoost:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost:
Macro F1 Score:       0.9664
Micro F1 Score:       0.9799
Macro Precision:      0.9747
Micro Precision:      0.9790
Macro Recall:         0.9630
Micro Recall:         0.9808


# Проверка на тесовых данных

Интерпретация результатов 

In [309]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import MultiLabelBinarizer

test_data = pd.read_excel("KRGN-MK_GP07_Фасады.xlsx", sheet_name="Фасады_Секции по файлу")

#=== Clean data
test_tangle_data = test_data[['Unnamed: 2', 'Unnamed: 3']]
test_tangle_data = test_tangle_data.dropna()

test_tangle_data = test_tangle_data[test_tangle_data['Unnamed: 2'].str.count(r"\.") > 1]

test_tangle_data.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
test_tangle_data = test_tangle_data.reset_index(drop=True)
test_tangle_data['ElementId'] = test_tangle_data['ElementId'].astype('int64')

tangle_groups = test_tangle_data.groupby('ElementId')['Position'].apply(list).reset_index()
#==== Concat all csv
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_['" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

test_revit_data = multi_concat()


"""
по началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать 
замеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях
что приведет к неправильному расчету метрик
"""



C:\Users\homer\AppData\Local\Temp\ipykernel_21520\2065048572.py:29: DtypeWarning: Columns (195,220,221,231,234,235,237,240,245,247,254,255,258,259,260,262,268,270,271,277,286,288,290,314,317,318,319,366,372,426,465,487,678,681,809,821,822,823,824,825,826,829,830,831,832,833,834,835,836,837,838,839,865,867,899,900,980,983,984,985,986,993,1090,1094,1095,1100,1143,1148,1160,1165,1166,1167,1168,1169,1172,1173,1174,1175,1176,1177,1178,1179,1180,1205,1217,1229,1505,1514,1806,1809,1812,1815,1835,1873,1874,1931,2106,2108,2129,2132,2135,2138,2141,2144) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
C:\Users\homer\AppData\Local\Temp\ipykernel_21520\2065048572.py:29: DtypeWarning: Columns (183,214,215,225,230,231,233,236,241,243,250,251,254,255,256,258,264,266,267,273,282,285,287,311,314,315,316,328,337,369,375,430,488,511,745,748,836,884,896,897,898,899,900,901,904,905,906,907,908,909,910,911,912,913,914,941,943,982,983,1057,1060,1061,1063,1064

'\nпо началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать \nзамеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях\nчто приведет к неправильному расчету метрик\n'

In [310]:
#=== Marge
# tangle_groups = tangle_groups.sample(frac=1, random_state=42).reset_index()
# del tangle_groups['index']
test_marged_df = tangle_groups.merge(test_revit_data, left_on='ElementId', right_on='ElementId')
# test_marged_df = test_marged_df.sample(frac=1, random_state=42).reset_index()

~~Преобразования датафреймов. y - tangle_groups, x - test_revit_data~~

In [364]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
X = test_marged_df.drop(columns=target_cols, axis=1)
y = test_marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_fit = mlb.fit_transform(y['Position'])
y = pd.DataFrame(y_fit, columns=mlb.classes_,index=y['ElementId'])
y = y.add_prefix('Target_')

with open('features.json', 'r') as f:
    features = json.load(f)

#=== Выбор только нужных фичей
X = X[X.columns.intersection(features)]

# constant_cols = X.columns[X.nunique() <= 1]
# X = X.drop(columns=constant_cols)
# print("Константные колонки:", len(list(constant_cols)))

X.fillna("NAN", inplace=True)

C:\Users\homer\AppData\Local\Temp\ipykernel_21520\3192538005.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NAN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  X.fillna("NAN", inplace=True)


In [136]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. 
Надо занулить лишние таргеты, что бы занулить вероятности при обучении.
"""
tangle_groups.columns.__len__() # 31 таргеты на тестовых данных 
y_df.columns.__len__() # 29 таргеты на обучающих данных 


29

In [365]:
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib

lgbm = LGBMClassifier(
    random_state=42
)

catb = CatBoostClassifier(
    random_state=42,
    nan_mode="Min"
)

multi_lgbm = MultiOutputClassifier(lgbm)

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)

for col in test_cat_features:
    try:
        X[col] = X[col].astype(str)
        """
        Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
        всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
        заполняются специальным пропуском. 
        """
        X[col] = X[col].astype('category')
    except KeyError:
        print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
        X[col] = 'missing'
        # test_cat_features.remove(col)
        continue  # или ничего не делать

multi_catb = MultiOutputClassifier(catb)

#=== Загрузка
print("Загрузка LightGBM и CatBoost завершена")
multi_lgbm = joblib.load('lgbm.pkl')
multi_catb = joblib.load('catb.pkl')


Загрузка LightGBM и CatBoost завершена


In [401]:
# Предсказание вероятностей
y_pred_proba_lgbm = np.array([clf.predict_proba(X)[:,1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_catb = np.array([clf.predict_proba(X)[:,1] for clf in multi_catb.estimators_]).T

# усреднение вероятностей
y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

# отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_lgbm > 0.5).astype(int)

In [402]:
y_pred_ensemble

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [403]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. y_pred_ensemble.shape - (220244, 29)
Надо занулить лишние таргеты, что бы занулить то, что не предсказано, что бы не ломать метрики. но
У меня сейчас супер мало данных. Так что придется проверять только то что есть. 
"""
# Найдём, какие trained_targets присутствуют в y
expected_targets = y.columns.tolist()
with open('trained_targets.json', 'r') as f:
    trained_targets = json.load(f)

common_targets = [t for t in trained_targets if t in y.columns]
print(f"Общие таргеты: {len(common_targets)} из {len(trained_targets)}")

# Удаляем из y_pred_ensemble столбцы, соответствующие trained_targets, которых нет в y
# То есть находим индексы в trained_targets, где targ НЕ в y.columns
indexes_to_remove = [i for i, t in enumerate(trained_targets) if t not in y.columns]

# Удаляем эти столбцы из предсказаний
new_pred = np.delete(y_pred_ensemble, indexes_to_remove, axis=1)

# Оставляем в y только общие столбцы, и в том же порядке, как в trained_targets
new_y = y[common_targets]  # Это важно: порядок как в trained_targets, но только существующие

Общие таргеты: 15 из 29


In [390]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. y_pred_ensemble.shape - (220244, 29)
Надо занулить лишние таргеты, что бы занулить то, что не предсказано, что бы не ломать метрики. но
У меня сейчас супер мало данных. Так что придется проверять только то что есть. 
"""
tangle_groups.columns.__len__() # - 37 таргеты на тестовых данных 
y_df.columns.__len__() # - 29 таргеты на обучающих данных 

expected_targets = y.columns.tolist()
with open('trained_targets.json', 'r') as f:
    trained_targets = json.load(f)
trained_targets
print(expected_targets)
print(trained_targets)

indexes = []
data_targets = []
for ind, targ in enumerate(trained_targets):
    if targ not in expected_targets:
        # print(targ)
       indexes.append(ind)
       data_targets.append(targ)
indexes
new_pred = result = np.delete(y_pred_ensemble, indexes, axis=1)
new_y = y.drop(data_targets)


['Target_004.014.033.001.002', 'Target_004.014.033.001.005', 'Target_004.014.033.001.008', 'Target_004.014.033.001.012', 'Target_004.014.033.001.013', 'Target_004.014.033.001.015', 'Target_004.014.033.001.3', 'Target_004.014.033.002.001', 'Target_004.014.033.002.002', 'Target_004.014.033.002.003', 'Target_004.014.033.002.004', 'Target_004.014.033.002.005', 'Target_004.014.033.004.001', 'Target_004.014.033.004.002', 'Target_004.014.033.004.003', 'Target_004.014.033.004.004', 'Target_004.014.033.004.005', 'Target_004.014.033.004.006', 'Target_004.014.033.006.001.1', 'Target_004.014.033.006.002', 'Target_021.004.003', 'Target_021.005.003.005.073', 'Target_021.005.003.005.074', 'Target_021.005.003.005.075', 'Target_021.005.003.005.082', 'Target_021.015.001.017', 'Target_031.003.002.006', 'Target_031.003.003.001', 'Target_031.003.003.002', 'Target_031.003.003.003', 'Target_031.003.003.004', 'Target_031.003.003.005', 'Target_031.006.003', 'Target_031.006.004', 'Target_031.006.006', 'Target_0

KeyError: "['Target_004.014.033.003.002' 'Target_021.005.003.005.028'\n 'Target_021.005.003.005.037' 'Target_021.005.003.005.049'\n 'Target_021.005.003.005.050' 'Target_021.005.003.005.051'\n 'Target_021.005.003.005.052' 'Target_021.005.003.005.053'\n 'Target_021.005.003.005.054' 'Target_021.005.003.005.059'\n 'Target_021.005.003.005.063' 'Target_031.003.002.001'\n 'Target_031.003.002.002' 'Target_031.003.002.003'] not found in axis"

In [363]:
y.shape, final_pred.shape, pred_df.shape

((11446, 15), (11446, 29), (11446, 29))

In [408]:
# тут сравнения по 2-м датафреймам
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(new_y,new_pred, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(new_y, new_pred)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(new_y, new_pred, average='macro')
f1_micro = f1_score(new_y, new_pred, average='micro')

precision_macro = precision_score(new_y, new_pred, average='macro', zero_division=0)
precision_micro = precision_score(new_y, new_pred, average='micro', zero_division=0)

recall_macro = recall_score(new_y, new_pred, average='macro', zero_division=0)
recall_micro = recall_score(new_y, new_pred, average='micro', zero_division=0)

#=== Метрики
# print("Результаты ансамбля на тестовых данных LGBM + CatBoost:")
print("Результаты ансамбля LGBM + CatBoost на тестовых данных:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost на тестовых данных:
Macro F1 Score:       0.2129
Micro F1 Score:       0.2853
Macro Precision:      0.3266
Micro Precision:      0.3027
Macro Recall:         0.2243
Micro Recall:         0.2698


По метрикам видно, что модель переобучилась. Нужно бороться с переобучением.